In [64]:
import numpy as np
import qutip as qt
from scipy.linalg import logm, expm
from qiskit.quantum_info import Operator, state_fidelity
from qiskit import QuantumCircuit
from copy import deepcopy

from tools.classical import *
from tools.quantum import *

np.printoptions(precision=3, suppress=True)


#### Qutip Exact Hamiltonian

In [65]:
np.random.seed(667)
num_qubits = 4
num_energy_bits = 8
bohr_bound = 2 ** (-num_energy_bits + 1)
eps = 0.1
sigma = 5
eig_index = 1
T = 1
total_time = T * 64
num_trotter_steps = 100

hamiltonian = find_ideal_heisenberg(num_qubits, bohr_bound, eps, signed=False, for_oft=True)
rescaled_coeff = hamiltonian.rescaled_coeffs
# Corresponding Trotter step circuit
hamiltonian.trotter_step_circ = trotter_step_heisenberg(num_qubits, coeffs=rescaled_coeff, symbreak=True)

Original spectrum:  [-6.8015 -4.4489 -4.     -2.1796 -1.5341 -1.3091 -0.6806  0.      0.6806
  1.02    1.3091  1.8956  2.1796  3.327   4.4489  6.0929]
Ideal spectrum:  [0.     0.0821 0.0978 0.1613 0.1838 0.1917 0.2136 0.2374 0.2611 0.273
 0.2831 0.3035 0.3134 0.3535 0.3926 0.45  ]
Nonrescaled coefficients:  [0.9 0.6 1.  1. ]
Rescaled coefficients:  [0.03140896 0.02093931 0.03489884 0.03489884]


### Qutip unitary evolution construction and comparison: Exact vs Trotterized

$n = 3, k = 4$
Hamiltonian is rescaled already by its coupling strength, not by the rescaled time anymore (bit easier to have an overview)
$$H = 0.125\times Heisenberg_3 + Z_{bulk}$$

In [66]:
step_size = total_time / num_trotter_steps

#* EXACT UNITARIES
exact_U_qt = qt.Qobj(expm(1j * hamiltonian.qt * total_time))

#* QUTIP TROTTER
trott_step_qt = trotter_step_heisenberg_qt(num_qubits, step_size, coeffs=hamiltonian.rescaled_coeffs, symbreak = True)
trott_U_qt = ham_evol_qt(num_qubits, trott_step_qt, num_trotter_steps)
total_shift = total_time * hamiltonian.shift
shifted_trott_U_qt = qt.Qobj(expm(1j * total_shift * np.eye(2**num_qubits))) * trott_U_qt  


#* QUTIP DISTANCES
dist = np.linalg.norm(exact_U_qt.full() - shifted_trott_U_qt.full())
print(f'Distance between Qutip Trotter vs Exact {dist}')
fro_dist_exact_vs_trott_qt = qt.Qobj(exact_U_qt - shifted_trott_U_qt).norm('fro')
print(f'Frobenius dist between Qutip Trotter vs Exact {fro_dist_exact_vs_trott_qt}')


Distance between Qutip Trotter vs Exact 2.803443040366976
Frobenius dist between Qutip Trotter vs Exact 2.8034430403669766


### Observable expectation comparison: Exact vs Trotterized, finding a good number of Trotter steps

In [67]:
#* Initial state
randstate_better = np.zeros(2**num_qubits)
randstate_better[np.random.choice(2**num_qubits, 2**num_qubits//2, replace=False)] = 1
randstate_better /= np.linalg.norm(randstate_better)

# initial_energy = rand_ini_state.conj().T @ rescaled_hamiltonian_qt.data @ rand_ini_state
# print(f'Initial energy {initial_energy}')

# # Pretty garbage:
# # rand_state0 = np.random.random(2**num_qubits) + 1j * np.random.random(2**num_qubits)  # Random state
# # rand_state0 /= np.linalg.norm(rand_state0)
# # rand_state1 = np.random.random(2**num_qubits) + 1j * np.random.random(2**num_qubits)  # Random state
# # rand_state1 /= np.linalg.norm(rand_state1)
# # print(qt.fidelity(qt.Qobj(randstate_better), qt.Qobj(rand_state1)))

# #* Evolved states
# exact_evolved_state = qt.Qobj(exact_U_qt) * qt.Qobj(rand_ini_state_qt)
# trott_evolved_state = qt.Qobj(trott_U_qt) * qt.Qobj(rand_ini_state_qt)

# #* Fidelities
# exact_evolved_fidelity = qt.fidelity(exact_evolved_state, trott_evolved_state)
# # print('Shifted:')  #* Global phase won't show up in fidelity
# # fid_of_states_shifted = qt.fidelity(exact_evolved_state_shifted, trott_evolved_state_shifted)
# print(f'Exact-Evolved fidelity {exact_evolved_fidelity}')


#### Circuit

In [68]:
shift_allq = expm(1j * total_shift * np.eye(2**num_qubits))
ham_time_evol = ham_evol(num_qubits, trotter_step=hamiltonian.trotter_step_circ,
                         num_trotter_steps=num_trotter_steps, time=total_time)

circ = QuantumCircuit(num_qubits, name="H")

circ.append(Operator(shift_allq), range(num_qubits))
circ.compose(ham_time_evol, range(num_qubits), inplace=True)

print(f'#Qubits {num_qubits}')
print(f'#Trotter steps {num_trotter_steps}')
print(f'#Gates {len([(op, _, _) for (op, _, _) in circ.data])}')
# print(circ)


#Qubits 4
#Trotter steps 10
#Gates 141


### Comparing Circuit unitary operator to Exact

In [69]:
#* Circuit operators
trott_evol_circ_op = Operator(circ).data

#* Distance to exact unitary
dist_circ_vs_exact = np.linalg.norm(trott_evol_circ_op - exact_U_qt)
print(f'Distance between circ unitary and exact {dist_circ_vs_exact}')

# #* Distance to qutip trotter
dist_circ_vs_qutip_trotter = np.linalg.norm(trott_evol_circ_op - shifted_trott_U_qt.full())
print(f'Distance between circ unitary and qutip trotter {dist_circ_vs_qutip_trotter}')
frob_dist_circ_vs_qutip_trotter = np.linalg.norm(trott_evol_circ_op - shifted_trott_U_qt.full(), 'fro')
print(f'Frobenius distance between circ unitary and qutip trotter {frob_dist_circ_vs_qutip_trotter}')
# circ_qutip_dag = np.abs((qt.Qobj(trotterized_evolution_unitary) * qt.Qobj(trott_U_qt).dag()).full()) - qt.qeye(trotterized_evolution_unitary.shape[0])

# print(f'Distance between circ unitary and exact {dist_circ_unitary_vs_exact}')
# print(f'Distance between circ unitary and qutip trotter {dist_circ_vs_qutip_trotter}')
# print(circ_qutip_dag)
# print(np.linalg.norm(circ_qutip_dag))

Distance between circ unitary and exact 2.8034430403669757
Distance between circ unitary and qutip trotter 5.838143449046149e-15
Frobenius distance between circ unitary and qutip trotter 5.838143449046149e-15


### Fidelity of Circuit vs Exact

In [70]:
# circ_with_ini_state = QuantumCircuit(num_qubits, name="H")
# circ_with_ini_state.initialize(rand_ini_state, range(num_qubits))  #! Qiskit and Qutip order is not reversed for the state!
# initial_state_from_circ = Statevector(circ_with_ini_state).data
# energy_initial_state_from_circ_np = initial_state_from_circ.conj().T @ rescaled_hamiltonian_qt.data @ initial_state_from_circ
# print('Energy of initial state:')
# print(energy_initial_state_from_circ_np.real)

# circ_with_ini_state.compose(ham_time_evol, range(num_qubits), inplace=True)
# qiskit_end_state = Statevector(circ_with_ini_state).data  # Circ to state

# print('Energy after time evolution (should not be changed):')
# end_energy = qiskit_end_state.conj().T @ rescaled_hamiltonian_qt.data @ qiskit_end_state
# print(end_energy.real)

# print('QISKIT CIRCUIT FIDELITY')
# trott_evolved_state = Statevector(trott_evolved_state.full())
# exact_evolved_state = Statevector(exact_evolved_state.full())

# print(f'To qutip exact {state_fidelity(exact_evolved_state, qiskit_end_state)}')
# print(f'To qutip Trotter {state_fidelity(trott_evolved_state, qiskit_end_state)}')


### Trotterized Hamiltonian comparison, Qiskit vs Exact, Qutip vs Exact

In [71]:
#* LOGM
# logU = logm(trotterized_evolution_unitary)  # log U = i 2pi H T / rescaling_factor  #! It's not unique if any eigenenergy < 0
# trotterized_ham_from_circuit = logU / (1j * T) #- shift * qt.qeye(hamiltonian_qt.shape[0])
# qiskit_trott_ham_spec = np.linalg.eigvalsh(trotterized_ham_from_circuit)
# spectrum_dist = np.linalg.norm(qiskit_trott_ham_spec - exact_spec)
# print(f'Spectrum distance {spectrum_dist}')

#### Qiskit gate angle convention! RXX(-2x) = exp(x)

In [72]:
c = QuantumCircuit(2)
c.rxx(np.pi/2, 0, 1)
op = Operator(c)
print(op.data)
X = qt.sigmax()
XX = qt.tensor(X, X)
op_qt = expm(-1j*np.pi* XX / 4)
print(op_qt)


[[0.70710678+0.j         0.        +0.j         0.        +0.j
  0.        -0.70710678j]
 [0.        +0.j         0.70710678+0.j         0.        -0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        -0.70710678j 0.70710678+0.j
  0.        +0.j        ]
 [0.        -0.70710678j 0.        +0.j         0.        +0.j
  0.70710678+0.j        ]]
[[0.70710678+0.j         0.        +0.j         0.        +0.j
  0.        -0.70710678j]
 [0.        +0.j         0.70710678+0.j         0.        -0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        -0.70710678j 0.70710678+0.j
  0.        +0.j        ]
 [0.        -0.70710678j 0.        +0.j         0.        +0.j
  0.70710678+0.j        ]]
